In [1]:
from pdf2image import convert_from_path
import os
import pytesseract
import fitz  # PyMuPDF
import openai
import pandas as pd
import time  # import time module for pausing
import json
from dotenv import load_dotenv


In [2]:
if load_dotenv():
    print(".env geladen")
else:
    print("did not find a .env file")

.env geladen


In [ ]:

# Set your OpenAI API key here
# openai.api_key = 'sk-proj-Ue2xY2FVsfuh1aNNTCg9T3BlbkFJQZgtifQyraq93mqfkMf3'
openai_api_key = os.getenv('openaikey')


In [21]:

# Set up tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
os.environ['TESSDATA_PREFIX'] = r'C:/Program Files/Tesseract-OCR/tessdata'  # Update this path to your Tesseract tessdata directory


In [23]:

def extract_text_from_pdf(pdf_path, timeout=60):
    doc = fitz.open(pdf_path)
    text = ""
    start_time = time.time()
    for page in doc:
        if time.time() - start_time > timeout:
            print(f"Timeout reached for {pdf_path}")
            break
        text += page.get_text()
    return text

def extract_text_from_images(pdf_path, poppler_path=r'C:/Program Files/poppler/Release-24.02.0-0/poppler-24.02.0/Library\bin'):
    if poppler_path:
        images = convert_from_path(pdf_path, poppler_path=poppler_path)
    else:
        images = convert_from_path(pdf_path)
    
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='deu')  # Use 'deu' for German texts
    return text

def extract_data_from_text(text, max_text_size=15000):
    text = text[:max_text_size]
    prompt = '''Du bekommst gleich einen String, der mit OCR aus einer Rechnung extrahiert wurde. Es handelt sich um eine einzelne Rechnung, die auch aus mehreren Seiten bestehen kann. Extrahiere die folgenden Daten aus diesem Text und gib sie im JSON-Format für die gesamte Rechnung aus. Gebe nur ein einzelnes JSON-Objekt aus. Hier ist eine Erklärung dazu, wie das JSON Objekt aussehen soll und was in die Felder rein soll: {
      "Vertragsnummer": "Hier soll die Vertragsnummer stehen. Diese ist in der Regel ein Integer, könnte aber in Ausnahmefällen auch andere Zeichen enthalten",
      "Adresse Verbrauchsstelle": "Das hier ist die Adresse der Verbrauchsstelle. Verwechsele sie nicht mit der Rechnungsadresse! Sie besteht aus Straße, Hausnummer, Postleitzahl und Ort",
      "Verbrauchte Menge": "Hier soll die verbrauchte Menge in Kilowattstunden (kWh) stehen. Wenn die Menge in Megawattstungen (MWh) angegeben ist, multipliziere mal 1000",
      "Start abgerechneter Zeitraum": "Gebe hier den Anfang des Abgerechneten Zeitraums als JJJJ-MM-TT an",
      "Ende abgerechneter Zeitraum": "Gebe hier das Ende des Abgerechneten Zeitraums als JJJJ-MM-TT an",
      "Energieart": "Hier soll entweder Gas oder Strom stehen",
      "Fehler": "Gib hier an, ob du bestimmte Daten nicht gefunden hast. Gib dann True an, ansonsten False. Damit wollen wir fehlerhafte Daten filtern"
      }
      Bitte gebe nur den JSON-String aus. Achte darauf, dass nur ein JSON-Objekt ausgegeben wird. Wenn ein Wert nicht gefunden wird, dann gib "nicht gefunden" an. Wenn Daten doppelt gefunden werden, dann fasse die Ergebnisse zusammen und gib nur den besten Treffer aus.'''

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": text}],
        max_tokens=150
    )
    extracted_data = response.choices[0].message['content'].strip()
    return extracted_data  # Remove any extra newlines at the end

def categorize_invoice(pdf_path):
    keywords_electricity = ["Strom", "kWh", "Netzbetreiber", "Energie"]
    keywords_gas = ["Gas", "m³", "Heizkosten", "Gasanbieter", "therm", "brennwert", "kWh/m³", "Gasverbrauch", "Gasrechnung"]

    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()

    text = text.lower()

    electricity_score = sum(keyword.lower() in text for keyword in keywords_electricity)
    gas_score = sum(keyword.lower() in text for keyword in keywords_gas)

    if electricity_score > gas_score:
        return "Strom"
    elif gas_score > electricity_score:
        return "Gas"
    else:
        return "Nicht eindeutig"

def process_pdfs(pdf_folder):
    extracted_data = []

    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

    for file_path in pdf_files:
        text = extract_text_from_pdf(file_path)
        if not text.strip():
            text = extract_text_from_images(file_path)  # Use OCR if no text was found
        data = extract_data_from_text(text)
        print(f"Extracted data from text for {file_path}:\n{data}\n")  # Debugging output
        extracted_data.append(data)

        time.sleep(5)  # Pause for 5 seconds between requests to avoid rate limiting

    return extracted_data

def extract_dict_from_response(response):
    response = response.split('{',1)[1].split('}',1)[0]
    response = '{' + response + '}'
    return json.loads(response)

def create_excel_file(data, output_file_path):
    data_dict = {'Vertragsnummer': [], 'Adresse Verbrauchsstelle': [], 'Verbrauchte Menge': [], 'Start abgerechneter Zeitraum': [], 'Ende abgerechneter Zeitraum':[], 'Energieart': [], 'Fehler':[]}

    for entry in data:
        temp_dict = extract_dict_from_response(entry)
        print(temp_dict)
        data_dict['Vertragsnummer'].append(temp_dict['Vertragsnummer'])
        data_dict['Adresse Verbrauchsstelle'].append(temp_dict['Adresse Verbrauchsstelle'])
        data_dict['Verbrauchte Menge'].append(temp_dict['Verbrauchte Menge'])
        data_dict['Start abgerechneter Zeitraum'].append(temp_dict['Start abgerechneter Zeitraum'])
        data_dict['Ende abgerechneter Zeitraum'].append(temp_dict['Ende abgerechneter Zeitraum'])
        data_dict['Energieart'].append(temp_dict['Energieart'])
        data_dict['Fehler'].append(temp_dict['Fehler'])

    df = pd.DataFrame(data_dict)
    print(df)
    df.to_excel(output_file_path, index=False)
    print(f'Excel file created: {output_file_path}')

if __name__ == "__main__":
    pdf_folder = '../data'  # Folder where the PDF files are located

    if not os.path.exists(pdf_folder):
        print(f"The folder '{pdf_folder}' does not exist.")
    else:
        # Process PDFs and extract data
        extracted_data = process_pdfs(pdf_folder)

        # Create and save the Excel file
        output_file_path = 'extracted_data.xlsx'
        create_excel_file(extracted_data, output_file_path)


Extracted data from text for ../data\R061023000112.pdf:
{
      "Vertragsnummer": "1201940779",
      "Adresse Verbrauchsstelle": "Berufsbildungswerk Neumünster, Am Hohrkamp 54, D 24537 Neumünster",
      "Verbrauchte Menge": 9674,
      "Start abgerechneter Zeitraum": "2023-08-01",
      "Ende abgerechneter Zeitraum": "2023-08-31",
      "Energieart": "Strom",
      "Fehler": false
    }

Extracted data from text for ../data\R090124000157.pdf:
{
    "Vertragsnummer": "nicht gefunden",
    "Adresse Verbrauchsstelle": "Norddeutsche Gesellschaft für Diakonie mbH, c/o ENOPLAN GmbH, Theodor-Schäfer-Straße 14-26, 25813 Husum",
    "Verbrauchte Menge": 437045,
    "Start abgerechneter Zeitraum": "2023-11-01",
    "Ende abgerechneter Zeitraum": "2023-11-30",
    "Energieart": "Gas",
    "Fehler": false
}

Extracted data from text for ../data\R160623000581.pdf:
{
    "Vertragsnummer": 6238563,
    "Adresse Verbrauchsstelle": "Rungestr. 1, 24537 Neumünster Gartenstadt",
    "Verbrauchte Menge":